# Visualise the pre-trained agent in action

Modify the path to the weights and run the notebook.

In [ ]:
WEIGHTS = './weights/weights-[NUMBER]'  # NB: without '.pth'

In [ ]:
import torch
import imageio
import gymnasium as gym

from upload.agent import VisionDeepQ

In [ ]:
network = {
    "input_channels": 1, "outputs": 5,
    "channels": [32, 64, 64],
    "kernels": [5, 3, 3],
    "strides": [3, 2, 1],
    "nodes": [64]
}
optimizer = {
    "optimizer": torch.optim.AdamW,
    "lr": 0.001,
    "hyperparameters": {}
}

In [ ]:
value_agent = VisionDeepQ(
    network=network, optimizer=optimizer,
)

weights = torch.load(f'{WEIGHTS}.pth', map_location=torch.device('cpu'))
value_agent.load_state_dict(weights)

In [ ]:
environment = gym.make('ALE/Tetris-v5', render_mode="rgb_array",
                       obs_type="grayscale", frameskip=4, repeat_action_probability=0.25)
environment.metadata["render_fps"] = 30

In [ ]:
state = torch.tensor(environment.reset()[0], dtype=torch.float32).view((1, 1, 210, 160))

images = []
TERMINATED = TRUNCATED = False
while not (TERMINATED or TRUNCATED):
    action = value_agent(state).argmax(1).item()

    state, reward, TERMINATED, TRUNCATED, _ = environment.step(action)
    state = torch.tensor(state, dtype=torch.float32).view((1, 1, 210, 160))

    images.append(environment.render())
_ = imageio.mimsave(f'./{WEIGHTS}.gif', images, duration=25)